In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader

import lightning as L
from LSTM import LSTMbyHand
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

# Preprocessing & Load data

In [ ]:
model = LSTMbyHand()
scaler = MinMaxScaler()

df = pd.read_csv('/Bitcoin 2024.csv', sep=';')
df['date'] = pd.to_datetime(df['timeOpen'], format='ISO8601')

df = df.sort_index(ascending=False).reset_index()
df = df[['timeOpen','close']]

input_values_scaled = scaler.fit_transform(df[['close']])

input = input_values_scaled[:-1].flatten()
input = torch.tensor(input, dtype=torch.float32)
input_value = torch.tensor(input, dtype=torch.float32).unsqueeze(0)

label = input_values_scaled[-1]
label_value = torch.tensor(label, dtype=torch.float32)

print("Bitcoin: Observed = 68526.1016, Predicted =", model(input))

Bitcoin: Observed = 68526.1016, Predicted = tensor(-0.2471, grad_fn=<MulBackward0>)


/var/folders/qy/94yjrytd2xld3rt1bgznnqs00000gn/T/ipykernel_9743/1790551668.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_value = torch.tensor(input, dtype=torch.float32).unsqueeze(0)


In [12]:
dataset = TensorDataset(input_value, label_value)
dataloader = DataLoader(dataset) 

for batch in dataloader:
    inputs, targets = batch
    print("Batch Features:", inputs)
    print("Batch Targets:", targets)

Batch Features: tensor([[5.6149e-02, 5.1434e-02, 5.0057e-02, 6.6368e-02, 6.8768e-02, 9.0684e-02,
         8.7900e-02, 8.5987e-02, 8.6410e-02, 6.1863e-02, 7.4144e-02, 8.0937e-02,
         7.7630e-02, 9.1948e-02, 7.8817e-02, 6.9441e-02, 5.3579e-02, 5.2839e-02,
         5.2067e-02, 3.9119e-02, 3.5037e-02, 3.4601e-02, 3.7656e-02, 4.3121e-02,
         3.9867e-02, 4.7418e-02, 3.5604e-02, 3.6812e-02, 4.1805e-02, 3.3970e-02,
         3.6002e-02, 4.3809e-02, 2.5233e-02, 2.8181e-02, 3.3250e-02, 3.6358e-02,
         6.1740e-02, 5.4655e-02, 5.3748e-02, 4.3683e-02, 3.5349e-02, 4.4307e-02,
         4.0669e-02, 4.1585e-02, 1.3249e-02, 4.4082e-02, 2.5466e-02, 2.8848e-02,
         2.8268e-02, 1.5150e-02, 1.7004e-02, 1.6219e-02, 1.6557e-02, 0.0000e+00,
         9.4189e-03, 2.5080e-02, 2.8900e-02, 2.5262e-02, 3.5997e-02, 6.6783e-02,
         1.0223e-01, 9.9827e-02, 1.1616e-01, 1.1310e-01, 1.1167e-01, 1.0731e-01,
         1.1601e-01, 1.0345e-01, 1.1094e-01, 1.1161e-01, 1.1396e-01, 1.1460e-01,
         1.2

In [13]:
print("Bitcoin: Observed = 0.9050, Predicted =", model(input))

Bitcoin: Observed = 0.9050, Predicted = tensor(-0.2471, grad_fn=<MulBackward0>)


In [14]:
trainer = L.Trainer(max_epochs=200)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/maulanashendi/self-project/model-from-scratch/LSTM/uploaded/lightning_logs

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)
/Users/maulanashendi/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/maulanashendi/anaconda3/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the 

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [15]:
print("Bitcoin: Observed = 0.9050, Predicted =", model(input))

Bitcoin: Observed = 0.9050, Predicted = tensor(0.8908, grad_fn=<MulBackward0>)


In [17]:
path_to_best_chackpoint = trainer.checkpoint_callback.best_model_path
trainer = L.Trainer(max_epochs=400)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_chackpoint)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /Users/maulanashendi/self-project/model-from-scratch/LSTM/uploaded/lightning_logs/version_0/checkpoints/epoch=199-step=200.ckpt
/Users/maulanashendi/anaconda3/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:360: The dirpath has changed from '/Users/maulanashendi/self-project/model-from-scratch/LSTM/uploaded/lightning_logs/version_0/checkpoints' to '/Users/maulanashendi/self-project/model-from-scratch/LSTM/uploaded/lightning_logs/version_2/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainab

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=400` reached.


In [18]:
print("Bitcoin: Observed = 0.9050, Predicted =", model(input))

Bitcoin: Observed = 0.9050, Predicted = tensor(0.9050, grad_fn=<MulBackward0>)
